[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io/)

# Machine Learning Methods

## Supervised Learning - Decision Tree Classifier - Exercise Solution

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 04/02/2023 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/MachineLearningMethods/2023_01/0021DecisionTreeExerciseSolution.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.datasets import fetch_openml
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.tree import DecisionTreeClassifier

# Miscellaneous
import os
from platform import python_version
import random

# Typing
from typing import Tuple

# Visualization
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

In [ ]:
# Configuration
%matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF = (8, 8)
ELM_SIZE_DEF = 50
CLASS_COLOR = ('b', 'r')


In [ ]:
# Fixel Algorithms Packages


## Decision Tree

In this exercise we'll use the _Decision Tree_ model as a classifier.  
The SciKit Learn library implement it with the [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) class.  

We'll use the [Heart Disease Data Set](https://archive.ics.uci.edu/ml/datasets/Heart+Disease) (Also known as Cleveland Heard Disease).  
The data set contains binary and categorical features which Decision Trees excel utilizing.  

The data set has the following columns:

- `age`: Age in years.
- `sex`: Sex (`1`: male; `0`: female).
- `cp`: Chest pain type: {`0`: typical angina, `1`: atypical angina, `2`: non-anginal pain, `3`: asymptomatic}.
- `trestbps`: Resting blood pressure (in mm Hg on admission to the hospital).
- `chol`: Serum cholestoral in mg/dl.
- `fbs`: Check fasting blood sugar: {`1`: Above 120 mg/dl, `0`: Below 120 mg/dl}.
- `restecg`: Resting electrocardiographic results: {`0`: normal, `1`: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), `2`: showing probable or definite left ventricular hypertrophy by Estes' criteria}.
- `thalach`: Maximum heart rate achieved.
- `exang`: Exercise induced angina: {`1`: yes, `0`: no}.
- `oldpeak` = ST depression induced by exercise relative to rest.
- `slope`: The slope of the peak exercise ST segment: {`0`: upsloping, `1`: flat, `2`: downsloping}.
- `ca`: Number of major vessels (0-3) colored by flourosopy.
- `thal`: {`0`: normal, `1`: fixed defect, `2`: reversable defect}.
- `num`: The target variable: {`0`: `<50` (No disease), `1`: `>50_1` (disease)}.

The exercise will also show the process of handling real world data: Removing invalid data, mapping values, etc...

I this exercise we'll do the following:

1. Load the [Heart Disease Data Set](https://archive.ics.uci.edu/ml/datasets/Heart+Disease) using `fetch_openml()` with id .
2. Validate data.
3. Convert text to numerical data (Though still as categorical data).
4. Train a decision tree.
5. Optimize the parameters: `criterion` and `max_leaf_nodes` by the `f1` score.
6. Train the optimal model on all data.
7. Display the Confusion Matrix and extract the different types of predictions.
8. Show the feature importance rank of the model.

* <font color='brown'>(**#**)</font> In order to let the classifier know the data is binary / categorical we'll use a **Data Frame** as the data structure.

In [ ]:
# Parameters

#===========================Fill This===========================#
# 1. Set the options for the `criterion` parameter (Use all options).
# 2. Set the options for the `max_leaf_nodes` parameter.
lCriterion   = ['gini', 'entropy', 'log_loss']
lMaxLeaf     = list(range(5, 11))
#===============================================================#

In [ ]:
# Auxiliary Functions

def PlotLabelsHistogram(vY: np.ndarray, hA = None):

    if hA is None:
        hF, hA = plt.subplots(figsize = (8, 6))
    
    vLabels, vCounts = np.unique(vY, return_counts = True)

    hA.bar(vLabels, vCounts, width = 0.9, align = 'center')
    hA.set_xticks(vLabels)
    hA.set_title('Histogram of Classes / Labels')
    hA.set_xlabel('Class')
    hA.set_ylabel('Number of Samples')

    return hA

def PlotConfusionMatrix(vY: np.ndarray, vYPred: np.ndarray, normMethod: str = None, hA: plt.Axes = None, lLabels: list = None, dScore: dict = None, titleStr: str = 'Confusion Matrix') -> plt.Axes:

    # Calculation of Confusion Matrix
    mConfMat = confusion_matrix(vY, vYPred, normalize = normMethod)
    oConfMat = ConfusionMatrixDisplay(mConfMat, display_labels = lLabels)
    oConfMat = oConfMat.plot(ax = hA)
    hA = oConfMat.ax_
    if dScore is not None:
        titleStr += ':'
        for scoreName, scoreVal in  dScore.items():
            titleStr += f' {scoreName} = {scoreVal:0.2},'
        titleStr = titleStr[:-1]
    hA.set_title(titleStr)
    hA.grid(False)

    return hA, mConfMat
    

## Generate / Load Data


In [ ]:
# Loading / Generating Data

dfData, dsY = fetch_openml('heart-c', version = 1, return_X_y = True, as_frame = True, parser = 'auto')

print(f'The data shape: {dfData.shape}')

## Pre Process Data

In this section we'll transform the data into features which the algorithms can work with.

### Remove Missing / Undefined Values

There are 3 main strategies with dealing with missing values:
1. A model to interpolate them.
2. Remove the sample.
3. Remove the feature.

The choice between (2) and (3) depends on the occurrence of the missing values.  
If there is a feature which is dominated by missing values, we might want to consider remove it.  
Otherwise, we'll remove samples with missing values.

* <font color='brown'>(**#**)</font> In case of large data set we might even build different models to different combinations of features.
* <font color='brown'>(**#**)</font> If missing values can happen in production, we need to think of a strategy that holds in that case as well.
* <font color='brown'>(**#**)</font> In practice, another factor to take into account is the importance of the feature.

In [ ]:
# Null / NA / NaN Matrix

dfData['Positive'] = dsY #<! Merge data

#===========================Fill This===========================#
# 1. Calculate the logical map of invalid values using the method `isna()`.
dfInvData = dfData.isna() #<! The logical matrix (DF) of invalid values
#===============================================================#

hF, hA = plt.subplots(figsize = FIG_SIZE_DEF)
sns.heatmap(data = dfInvData, square = False, ax = hA)
hA.set_title('Invalid Data Map')

plt.show()


* <font color='red'>(**?**)</font> Given the results above, would you remove a feature or few samples?

In [ ]:
# Remove NaN / Null Values

#===========================Fill This===========================#
# 1. Remove the NaN / Null values. Use `dropna()`.
# !! Choose the correct policy (Remove samples or features) by `axis`.
dfX = dfData.dropna(axis = 0)
#===============================================================#

print(f'The features data shape: {dfX.shape}')

In [ ]:
# Drop Duplicate Rows

#===========================Fill This===========================#
# 1. Drop duplicate rows (Samples) using the method `drop_duplicates()`.
# 2. Reset index using the method `reset_index()` .
dfX = dfX.drop_duplicates()
dfX = dfX.reset_index(drop = True)
#===============================================================#
dfX = dfX.astype(dtype = {'ca': np.int64}) #<! It's integer mistyped as Float64

In [ ]:
# Split X, y

dsY = dfX['Positive']
dfX = dfX.drop(columns = ['Positive'])

In [ ]:
# Distribution of Labels

hA = PlotLabelsHistogram(dsY)
plt.show()

* <font color='red'>(**?**)</font> Is the data balanced?

### Convert Data into Numeric Values

Some of the categorical and binary data is given by text values.  
It is better to convert them into numerical values (Though some models can work with them as is).  
For some visualizations, the textual data is great, hence we keep it.


* <font color='brown'>(**#**)</font> Usually this is done as part of the pipeline. See `OneHotEncoder`, `Binarizer` and `OrdinalEncoder` in the `sklearn.preprocessing` module.
* <font color='brown'>(**#**)</font> Currently, the implementation of `DecisionTreeClassifier` doesn't support categorical values which are not ordinal. Hence we must use `OneHotEncoder`. See https://github.com/scikit-learn/scikit-learn/pull/12866.
* <font color='brown'>(**#**)</font> The _One Hot Encoding_ is not perfect. See [Are Categorical Variables Getting Lost in Your Random Forests](https://web.archive.org/web/20200307172925/https://roamanalytics.com/2016/10/28/are-categorical-variables-getting-lost-in-your-random-forests/) (Also [Notebook - Are Categorical Variables Getting Lost in Your Random Forests](https://notebook.community/roaminsight/roamresearch/BlogPosts/Categorical_variables_in_tree_models/categorical_variables_post)). 
* <font color='brown'>(**#**)</font> The SciKit Learn has support for categorical features in the `HistGradientBoostingClassifier` class.


In [ ]:
# See the Type of the Features
dfX.info()

In [ ]:
# Lists of the Features Type

lBinaryFeature  = ['sex', 'fbs', 'exang']
lCatFeature     = ['cp', 'restecg', 'slope', 'thal']
lNumFeature     = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'ca']

In [ ]:
# Creating a Copy (Numerical)

#===========================Fill This===========================#
# 1. Create a copy (Not a view) using the method `copy()`.
dfXNum = dfX.copy()
#===============================================================#

In [ ]:
# Encode Binary Categorical Features
# Usually this is done using `Binarizer` and `OrdinalEncoder`.
# Yet there is a defined mapping in the data description which will be used.

dSex        = {'female': 0, 'male': 1}
dCp         = {'typ_angina': 0, 'atyp_angina': 1, 'non_anginal': 2, 'asympt': 3}
dFbs        = {'f': 0, 't': 1}
dRestEcg    = {'normal': 0, 'st_t_wave_abnormality': 1, 'left_vent_hyper': 2}
dExAng      = {'no': 0, 'yes': 1}
dSlope      = {'up': 0, 'flat': 1, 'down': 2}
dThal       = {'normal': 0, 'fixed_defect': 1, 'reversable_defect': 2}

dMapper = {'sex': dSex, 'fbs': dFbs, 'exang': dExAng, 'cp': dCp, 'restecg': dRestEcg, 'slope': dSlope, 'thal': dThal}

for colName in (lBinaryFeature + lCatFeature):
    # dMapping = dMapper[colName]
    dfXNum[colName] = dfXNum[colName].map(dMapper[colName])


In [ ]:
# Encode the Labels

#===========================Fill This===========================#
# 1. Create a dictionary which maps the string `<50` to 0 and `>50_1` to 1.
# 2. Apply a mapping on `dsY` using the method `map()`.
dMapY   = {'<50': 0, '>50_1': 1}
dsY     = dsY.map(dMapY)
#===============================================================#

dsY = dsY.rename('Positive')

### Exploratory Data Analysis (EDA)

This is the stage we're trying to infer insights on the data using visualizations.  
This is a skill which requires experience and creativity.  

We'll do some very basic operations for this data set.  
We'll see the distribution of each feature for the 2 values.


In [ ]:
# Binary Data

numFeatures = len(lBinaryFeature)
hF, hA = plt.subplots(1, numFeatures, figsize = (24, 8))
hA = hA.flat

for ii, colName in enumerate(lBinaryFeature):
    sns.histplot(data = dfX, x = colName, hue = dsY, discrete = True, multiple = 'dodge', ax = hA[ii])

In [ ]:
# Categorical Data

numFeatures = len(lCatFeature)
hF, hA = plt.subplots(1, numFeatures, figsize = (24, 8))
hA = hA.flat

for ii, colName in enumerate(lCatFeature):
    sns.histplot(data = dfX, x = colName, hue = dsY, discrete = True, multiple = 'dodge', ax = hA[ii])

In [ ]:
# Numerical Data

lDiscreteData = []

numFeatures = len(lNumFeature)
hF, hA = plt.subplots(1, numFeatures, figsize = (24, 8))
hA = hA.flat

for ii, colName in enumerate(lNumFeature):
    # if pd.api.types.is_integer_dtype(dfX[colName]):
    #     sns.histplot(data = dfX, x = colName, hue = dsY, discrete = True, multiple = 'dodge', ax = hA[ii])
    if colName == 'ca':
        sns.histplot(data = dfX, x = colName, hue = dsY, discrete = True, multiple = 'dodge', ax = hA[ii])
    else:
        sns.kdeplot(data = dfX, x = colName, hue = dsY, fill = True, common_norm = True, ax = hA[ii])

* <font color='red'>(**?**)</font> How would you handle the case where a feature has a single value? Look at [`VarianceThreshold`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html#sklearn.feature_selection.VarianceThreshold).
* <font color='brown'>(**#**)</font> Usually part of the work on feature includes a process to select the best of them. For example a brute force method is given by [`SelectKBest`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html).

## Train a Decision Tree Model and Optimize Hyper Parameters

In this section we'll optimize the model according to the `F1` score.  
The `F1` score is the geometric mean of the _precision_ and _recall_.  
Hence it can handle pretty well imbalanced data as well (Though this case is not really that imbalanced).

We'll use the `f1_score()` function to calculate the measure.  
The process to optimize the _Hyper Parameters_ will be as following:

1. Build a data frame to keep the scoring of the different hyper parameters combination.
2. Optimize the model:
  - Construct a model using the current combination of _hyper parameters_.
  - Apply a cross validation process to predict the data using `cross_val_predict()`.
  - As the cross validation iterator (The `cv` parameter) use `KFold` to implement _Leave One Out_ policy.
3. Calculate the `F1` score of the predicted classes.
4. Store the result in the performance data frame.


* <font color='brown'>(**#**)</font> Pay attention that while we optimize the hyper parameters according to the `F1` score, the model itself has a different _loss_ function.

In [ ]:
# Creating the Data Frame

#===========================Fill This===========================#
# 1. Calculate the number of combinations.
# 2. Create a nested loop to create the combinations between the parameters.
# 3. Store the combinations as the columns of a data frame.

# For Advanced Python users: Use iteration tools for create the cartesian product
numComb = len(lCriterion) * len(lMaxLeaf)
dData   = {'Criterion': [], 'Max Leaves': [], 'F1': [0.0] * numComb}

for ii, paramCriteria in enumerate(lCriterion):
    for jj, maxLeaf in enumerate(lMaxLeaf):
        dData['Criterion'].append(paramCriteria)
        dData['Max Leaves'].append(maxLeaf)
#===============================================================#

dfModelScore = pd.DataFrame(data = dData)
dfModelScore



In [ ]:
# Optimize the Model

#===========================Fill This===========================#
# 1. Iterate over each row of the data frame `dfModelScore`. Each row defines the hyper parameters.
# 2. Construct the model.
# 3. Train it on the Train Data Set.
# 4. Calculate the score.
# 5. Store the score into the data frame column.

for ii in range(numComb):
    paramCriteria    = dfModelScore.loc[ii, 'Criterion']
    maxLeaf          = dfModelScore.loc[ii, 'Max Leaves']

    print(f'Processing model {ii + 1:03d} out of {numComb} with `criterion` = {paramCriteria} and `max_leaf_nodes` = {maxLeaf}.')

    oDecTreeCls = DecisionTreeClassifier(criterion = paramCriteria, max_leaf_nodes = maxLeaf)
    
    vYPred = cross_val_predict(oDecTreeCls, dfXNum, dsY, cv = KFold(n_splits = dfXNum.shape[0]))

    f1Score = f1_score(dsY, vYPred)
    dfModelScore.loc[ii, 'F1'] = f1Score
    print(f'Finished processing model {ii + 1:03d} with `f1 = {f1Score}.')
#===============================================================#

* <font color='red'>(**?**)</font> Should we use _Stratified K Fold_ above? Why?
* <font color='brown'>(**#**)</font> Pay attention to the speed of the process. This is one of the main advantages of models based on trees.

In [ ]:
# Display Sorted Results (Descending)
# Pandas allows sorting data by any column using the `sort_values()` method
# The `head()` allows us to see only the the first values
dfModelScore.sort_values(by = ['F1'], ascending = False).head(10)

* <font color='red'>(**?**)</font> Do you find results satisfactory? How would you answer this?  
   See the _Code 001_ below.
* <font color='green'>(**@**)</font> Since the class `DecisionTreeClassifier` doesn't really support categorical features, you may use _One Hot Encoding_.  
   See [Pandas' `get_dummies()`](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html). You may use _Code 002_ below.


```python
# Code 001
from sklearn.linear_model import LogisticRegression

oCls = LogisticRegression(max_iter = 10_000)
oCls = oCls.fit(dfXNum, dsY)
f1_score(dsY, oCls.predict(dfXNum))
```

```python
# Code 002
dfXCat = dfXNum.copy()

for colName in lCatFeature:
    dfTmp = pd.get_dummies(dfXNum[colName], prefix = colName)
    dfXCat = pd.concat((dfXCat, dfTmp), axis = 1)
    dfXCat = dfXCat.drop(columns = colName)

dfXCat
```


<!-- from sklearn.ensemble import HistGradientBoostingClassifier

oCls = HistGradientBoostingClassifier(max_iter = 100, max_leaf_nodes = 20, categorical_features = lBinaryFeature + lCatFeature)
oCls = oCls.fit(dfXNum, dsY)
f1_score(dsY, oCls.predict(dfXNum)) -->

In [ ]:
# Plotting the Train Data F1 as a Heat Map
# We can pivot the data set created to have a 2D matrix of the `F1` as a function of `Criterion` and the `Max Leaves`.

hA = sns.heatmap(data = dfModelScore.pivot(index = 'Max Leaves', columns = 'Criterion', values = 'F1'), robust = True, linewidths = 1, annot = True, fmt = '0.2%', norm = LogNorm())
hA.set_title('F1 of the Cross Validation')
plt.show()

In [ ]:
# Extract the Optimal Hyper Parameters

#===========================Fill This===========================#
# 1. Extract the index of row which maximizes the score.
# 2. Use the index of the row to extract the hyper parameters which were optimized.

#! You may find the `idxmax()` method of a Pandas data frame useful.
idxArgMax = dfModelScore['F1'].idxmax()
#===============================================================#

optimalCriterion = dfModelScore.loc[idxArgMax, 'Criterion']
optimalMaxLeaf   = dfModelScore.loc[idxArgMax, 'Max Leaves']

print(f'The optimal hyper parameters are: `criterion` = {optimalCriterion}, `max_leaf_nodes` = {optimalMaxLeaf}')



### Optimal Model

In this section we'll extract the best model an retrain it on the whole data (`dfXNum`).  
We need to export the model which has the best Test values.

In [ ]:
# Construct the Optimal Model & Train on the Whole Data

#===========================Fill This===========================#
# 1. Construct the model with the optimal hyper parameters.
# 2. Fit the model on the whole data set.
oDecTreeCls = DecisionTreeClassifier(criterion = optimalCriterion, max_leaf_nodes = optimalMaxLeaf)
oDecTreeCls = oDecTreeCls.fit(dfXNum, dsY)
#===============================================================#

In [ ]:
# Model Score (Accuracy)

print(f'The model score (Accuracy) is: {oDecTreeCls.score(dfXNum, dsY):0.2%}.')

In [ ]:
# Plot the Confusion Matrix
hF, hA = plt.subplots(figsize = (10, 10))

#===========================Fill This===========================#
hA, mConfMat = PlotConfusionMatrix(dsY, oDecTreeCls.predict(dfXNum), hA = hA)
#===============================================================#

plt.show()

* <font color='red'>(**?**)</font> Calculate the TP, TN, FP and FN rates. 
* <font color='red'>(**?**)</font> Calculate the _precision_ and _recall_.
* <font color='red'>(**?**)</font> Calculate the _precision_ and _recall_ assuming the labels `0` is the positive label.

## Feature Significance

One advantage of the decision tree based models is having access to the significance of each feature during training.  
We can access it using the `feature_importances_` attribute (Only after a applying training by the `fit()` method).

* <font color='brown'>(**#**)</font> This ability is useful as a pre processing of data for any model with no restriction to trees.
* <font color='brown'>(**#**)</font> The idea is measuring the total contribution of the feature to the reduction in loss.
* <font color='brown'>(**#**)</font> This is a good metric for importance mainly for categorical features. For features with high number of unique values (Continuous), it might be not as accurate.

In [ ]:
# Extract the Importance of the Features

#===========================Fill This===========================#
# Extract the feature importance using the `feature_importances_` attribute
vFeatImportance = oDecTreeCls.feature_importances_
#===============================================================#

The feature importance is normalized, hence we can display it like a discrete probability mass function.

In [ ]:
# Plot the Feature Importance

hF, hA = plt.subplots(figsize = (16, 8))
hA.bar(x = dfXNum.columns, height = vFeatImportance)
hA.set_title('Features Importance of the Model')
hA.set_xlabel('Feature Name')
hA.set_title('Importance')

plt.show()

* <font color='red'>(**?**)</font> How many non zero values could we have? Look at the number of splits.
* <font color='red'>(**?**)</font> What can be done with the features with low value?
* <font color='brown'>(**#**)</font> Can you explain what you see with the EDA phase plots?
* <font color='brown'>(**#**)</font> Pay attention, in the context of feature importance we may choose high number of splits even if it means overfit. It won't be a model for production, but will give a better view of the features.